# C4 model

Generation of a Maize C4 model ("MaizeCore") from a mass and proton balanced model of plant primary metabolism

### Imports

In [1]:
import pandas as pd
import sys
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction
sys.path.append("../Code")
from model_functions import *

#General Core Model
general_model = read_sbml_model("../Models/PlantCoreMetabolism_v2_0_0.xml")

### Curation: Adding Reactions

In [2]:
"""
Maize Biomass
"""
#Create the biomass reaction
reaction = Reaction('Maize_biomass_tx')
reaction.name = 'Maize biomass'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.

#Import the pandas dataframe

df = pd.read_csv("../Datasets/Biomass_no_nucleotide.csv")

#Get a list with the metabolite IDs from the generic model
met_list = []

for id in df.loc[:,"Ids"]:
    met = general_model.metabolites.get_by_id(id)
    met_list.append(met)

#Create a list with the coefficients
coef = list(df.loc[:,"Maize"])

#Add metabolites to the model: dictionary - keys are metabolites and values are the coefficients
for i in range(len(coef)):
    reaction.add_metabolites({met_list[i]:coef[i]})

general_model.add_reactions([reaction])

In [3]:
"""
Malate/Pyruvate Transporter
"""

MAL_c = general_model.metabolites.MAL_c
MAL_p = general_model.metabolites.MAL_p
PYRUVATE_c = general_model.metabolites.PYRUVATE_c
PYRUVATE_p = general_model.metabolites.PYRUVATE_p

#Create the biomass reaction
transporter = Reaction('PYR_MAL_pc')
transporter.name = 'Malate/Pyruvate Transport'
transporter.lower_bound = -1000.  # This is the default
transporter.upper_bound = 1000.

transporter.add_metabolites({
    PYRUVATE_p: -1.0,
    MAL_c: -1.0,
    PYRUVATE_c: 1.0,
    MAL_p: 1.0
})

general_model.add_reactions([transporter])


In [4]:
"""
Proton mediated pyruvate transporter
"""

Pyr_c = general_model.metabolites.PYRUVATE_c
H_c = general_model.metabolites.PROTON_c
Pyr_p = general_model.metabolites.PYRUVATE_p
H_p = general_model.metabolites.PROTON_p

#Create the biomass reaction
transporter = Reaction('Pyr_H_pc')
transporter.name = 'Proton Mediated Pyruvate Transport'
transporter.lower_bound = -1000.  # This is the default
transporter.upper_bound = 1000.

transporter.add_metabolites({
    Pyr_c: -1.0,
    H_c: -1.0,
    Pyr_p: 1.0,
    H_p: 1.0
})

general_model.add_reactions([transporter])

In [5]:
"""
Correct proton stoichiometry of PEP transporter
"""

H_c = general_model.metabolites.PROTON_c
H_p = general_model.metabolites.PROTON_p
PI_c = general_model.metabolites.Pi_c
aPI_c = general_model.metabolites.aPi_c
PI_p = general_model.metabolites.Pi_p
PEP_c = general_model.metabolites.PHOSPHO_ENOL_PYRUVATE_c
PEP_p = general_model.metabolites.PHOSPHO_ENOL_PYRUVATE_p

#Create the biomass reaction
transporter = Reaction('PPT_pc')
transporter.name = 'Phosphoenolpyruvate Transporter'
transporter.lower_bound = -1000.  # This is the default
transporter.upper_bound = 1000.

transporter.add_metabolites({
    PEP_p: -1.0,
    H_p: -1.0,
    PI_c: -0.7,
    aPI_c: -0.3,
    PEP_c: 1.0,
    H_c: 1.3,
    PI_p: 1.0
})

general_model.add_reactions([transporter])


### Initial constrains applied to the base model

In [6]:
"""
Generic constrains
"""
#Initial import/export constraints
set_bounds('CO2_tx', (-1000, 1000), general_model)
set_bounds('H2O_tx', (-1000, 1000), general_model)
set_bounds('NH4_tx', (0., 0.), general_model)
set_bounds('Pi_tx', (0, 1000), general_model)
set_bounds('SO4_tx', (0, 1000), general_model)
set_bounds('O2_tx', (-1000, 1000), general_model)
set_bounds('Sucrose_tx', (-1000, 0), general_model) #Exported but not imported
set_bounds('GLC_tx', (-1000, 0), general_model)
set_bounds('unlProtHYPO_c', (0, 0), general_model)

#Bounds for ATP
set_bounds('ATPase_tx', (0, 1000), general_model)

#Bounds for chloroplastic NADPH dehydrogenase and plastoquinol oxidase

set_bounds('Plastoquinol_Oxidase_p', (0, 0), general_model)

#NTT is only active at night
set_fixed_flux("ATP_ADP_Pi_pc", 0, general_model)

#No uncoupled pyruvate transport
set_bounds('PYRUVATE_pc', (0, 0), general_model)

#Blocking Extra Biomass reaction
set_bounds("Biomass_tx", (0,0), general_model)
set_bounds("AraCore_Biomass_tx", (0,0), general_model)
set_bounds("Phloem_output_tx", (0,0), general_model)

#Block Former PEP transport reaction
set_bounds("PEP_Pi_pc", (0,0), general_model)


## Generation of the C4 "MaizeCore" model

## Model Duplication

In [7]:
"""
Creating the C4 model
"""

from cobra import Model, Reaction, Metabolite

c4_model = Model('c4_model')

cell_types = ['M', 'B']

#duplicate metabolites
for m in general_model.metabolites:
    for cell in cell_types:
        m_dt = Metabolite('['+cell+']_'+m.id, name = m.formula, compartment = m.compartment)
        c4_model.add_metabolites([m_dt])

#duplicate reactions
for r_c3_obj in general_model.reactions:
    for cell in cell_types:
        r_c4_obj = Reaction('['+cell+']_'+r_c3_obj.id)
        r_c4_obj.name = r_c3_obj.name
        r_c4_obj.subsystem = r_c3_obj.subsystem
        r_c4_obj.bounds = r_c3_obj.bounds
        c4_model.add_reaction(r_c4_obj)
        r_c4_obj.add_metabolites({'['+cell+']_'+m_c3_obj.id: r_c3_obj.get_coefficient(m_c3_obj) for m_c3_obj in r_c3_obj.metabolites})



## Establishing exchange of cytosolic metabolites between cell types

In [8]:
"""
Exchange Reactions

All metabolites except Na, AD, DHF, TRXox, TRXrd, T6P and OMP have
corresponding metabolites in the plastid compartment of the generic model

"""

Na,F26BP, ORO, DHO, GABA, PRPP, AD, DHF, starch, TRXox, TRXrd, T6P, OMP, UMP, CTP, dADP, dCDP, dGDP, dUDP, dATP, dCTP, dGTP = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


no_transport_general = [ Na, "HS",
                        "FRUCTOSE_16_DIPHOSPHATE",  F26BP, "DPG", "PROTON", "ACETALD", "ACET", "5_10_METHENYL_THF",
                        "5_METHYL_THF", "HOMO_CYS", "ADENOSYL_HOMO_CYS", ORO, DHO,
                        GABA, "ACETYLSERINE", PRPP, AD, "THF", DHF , "ADENOSINE", "MALTOSE", "CO_A", "L_GLUTAMATE_5_P", "aL_GLUTAMATE_5_P"
                        "ACETYL_COA", "CELLULOSE", starch, TRXox, TRXrd, "L_GLUTAMATE_GAMMA_SEMIALDEHYDE", T6P, "S_ADENOSYLMETHIONINE",
                        "PPI", "L_DELTA1_PYRROLINE_5_CARBOXYLATE", "CARBON_DIOXIDE", "OXALACETIC_ACID", "HCO3",
                        "UTP","aUTP", "UDP","aUDP", "UDP_GLUCOSE", "ATP", "aATP","ADP", "aADP","AMP", "IMP", "aIMP","XANTHOSINE_5_PHOSPHATE", "aXANTHOSINE_5_PHOSPHATE",
                        "GTP", "aGTP","GDP","aGDP", "GMP","aGMP", "bGMP",OMP, UMP, CTP, "GDP", "aGDP","CDP", dADP,
                        dCDP, dGDP, dUDP, "DUMP", "DTMP",  "aDTDP", "GTP",
                        dATP, dCTP, dGTP, "DTTP" "aDTTP", "NAD", "NADH", "NADP", "NADPH"]

#add M/BS exchange reactions
L_r_transport = []
for m_c3_obj in general_model.metabolites:
    if m_c3_obj.id[-1:] == 'c' and m_c3_obj.id[:-2] not in no_transport_general and  m_c3_obj.id[-2:] != 'mc':
        r_c4_obj = Reaction('[MB]_'+m_c3_obj.id)
        r_c4_obj.name = '[MB]_'+m_c3_obj.id
        r_c4_obj.subsystem = 'Exchange'
        r_c4_obj.bounds = (-1000, 1000)
        c4_model.add_reaction(r_c4_obj)
        r_c4_obj.add_metabolites({'[M]_'+m_c3_obj.id: -1,'[B]_'+m_c3_obj.id: 1 })
        L_r_transport.append('[MB]_'+m_c3_obj.id)

### Establishment of Rubisco populations in the bundle sheath

In [9]:
"""
Rubisco - Second unconstrained Rubisco population
"""

#CONSTRAINT: Add external CO2 species to bundle sheath
#(the original CO2 species is treated as internal CO2)
m_list_CO_Ex= ['[B]_CARBON_DIOXIDE_ex_c','[B]_CARBON_DIOXIDE_ex_p']

for m_id in m_list_CO_Ex:
    m_obj = Metabolite(m_id)
    m_obj.compartment = m_id[-1]
    c4_model.add_metabolites(m_obj)

#CONSTRAINT: Copy reactions 'Tr_CO2h', 'RBC_h' and replace internal CO2 with external CO2 in the copied reactions
r_list_CO_Ex = ['CO2_pc', 'RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p']

for r_id in r_list_CO_Ex:
    r_obj = c4_model.reactions.get_by_id('[B]_'+r_id)
    r_obj_Ex = Reaction(r_obj.id+'_Ex')
    r_obj_Ex.name = r_obj.id+'_Ex'
    r_obj_Ex.subsystem = r_obj.subsystem
    r_obj_Ex.bounds = r_obj.bounds
    c4_model.add_reaction(r_obj_Ex)
    r_obj_Ex.add_metabolites({m_obj.id if not m_obj.id[:-2] == '[B]_CARBON_DIOXIDE' else '[B]_CARBON_DIOXIDE_ex'+m_obj.id[-2:]: r_obj.get_coefficient(m_obj)
                                  for m_obj in r_obj.metabolites})

#CONSTRAINT: CO2 exchange between mesophyll and bundle sheat
r_c4_obj = Reaction('[MB]_CO2_c')
r_c4_obj.name = '[MB]_CO2_c'
r_c4_obj.subsystem = 'Exchange'
r_c4_obj.bounds = (-1000, 1000)
c4_model.add_reaction(r_c4_obj)
r_c4_obj.add_metabolites({'[M]_CARBON_DIOXIDE_c': -1,'[B]_CARBON_DIOXIDE_ex_c': 1 })
L_r_transport.append('[MB]_CARBON_DIOXIDE_c')

### Constrains specific to the "MaizeCore" C4 model

In [10]:
"""
C4 specific constrains
"""

#CONSTRAINT: No CO2 uptake in bundle sheat cells due to suberin layer in cell membranes
set_bounds("[B]_CO2_tx", (0.,0.), c4_model)

#Gaseous exchanges only through the M cell
set_bounds("[B]_O2_tx", (0.,0.,), c4_model)

# Only allow uptakes through the BS of inorganic metabolites
set_bounds("[M]_Nitrate_tx", (0.,0.), c4_model)
set_bounds("[M]_SO4_tx", (0.,0.), c4_model)
set_bounds("[M]_H2O_tx", (0.,0.), c4_model)
set_bounds("[M]_Ca_tx", (0.,0.), c4_model)
set_bounds("[M]_Mg_tx", (0.,0.), c4_model)
set_bounds("[M]_H2O_tx", (0.,0.), c4_model)
set_bounds("[M]_Pi_tx", (0.,0.), c4_model)
set_bounds("[M]_Ca_tx", (0.,0.), c4_model)
set_bounds("[M]_K_tx", (0.,0.), c4_model)

#Force NADP-ME decarboxylation pathway: Block all other decarboxylation reactions except NADP_ME in the plastid
set_fixed_flux("[B]_MALIC_NADP_RXN_c", 0, c4_model)

#Force NADP-ME decarboxylation pathways: make alternative decarboxylation routes irreversible
set_bounds('[B]_CARBAMATE_KINASE_RXN_p', (0, 1000), c4_model)
set_bounds('[M]_CARBAMATE_KINASE_RXN_p', (0, 1000), c4_model)

set_bounds('[B]_UREASE_RXN_c', (0, 0), c4_model)
set_bounds('[M]_UREASE_RXN_c', (0, 99999999), c4_model)

set_bounds('[B]_ISOCITDEH_RXN_m', (0, 1000), c4_model)
set_bounds('[M]_ISOCITDEH_RXN_m', (0, 1000), c4_model)

set_bounds('[B]_ISOCITDEH_RXN_c', (0, 1000), c4_model)
set_bounds('[M]_ISOCITDEH_RXN_c', (0, 1000), c4_model)

set_bounds('[B]_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m', (0, 1000), c4_model)
set_bounds('[M]_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m', (0, 1000), c4_model)

set_bounds('[M]_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c', (0, 0), c4_model)

#Block PSII in the BS cell
set_bounds('[B]_PSII_RXN_p', (0, 0), c4_model)

###Make 1.18.1.2 irreversible - Ferrodoxin:NADP(H) oxireductase (FNR)
set_bounds("[B]_1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p", (0, 1000), c4_model)
set_bounds("[M]_1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p", (-1000, 1000), c4_model)


In [11]:
general_model

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x02b5062e83c8
Number of metabolites,861
Number of reactions,896
Number of groups,208
Objective expression,1.0*Phloem_output_tx - 1.0*Phloem_output_tx_reverse_990b1
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


### Writing and saving the models

In [12]:
"""
Write the models

"""

write_sbml_model(general_model, "../Models/c3_model.xml")

write_sbml_model(c4_model, "../Models/c4_model.xml")
